In [88]:
from sympy import pi
from classiq import RX, H, QBit, QCallable, QFunc, QParam, allocate, create_model, symbolic, Model

model = Model()

@QFunc
def foo(
    n: QParam[int],
    my_operand: QCallable[QParam[float]],
    qb: QBit,
) -> None:
    H(target=qb)
    my_operand((float(pi) / n))


@QFunc
def main() -> None:
    qb1 = QBit("qb1")
    qb2 = QBit("qb2")
    allocate(1, qb1)
    allocate(1, qb2)
    foo(n=4, my_operand=lambda t: RX(theta=t, target=qb1), qb=qb2)

qmod = create_model(main)

qprog= synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/55a61598-42f6-40ab-8e1e-c1b59a51304d?version=0.36.1


In [7]:
import sympy

from classiq.builtin_functions.exponentiation import PauliOperator
from classiq.builtin_functions import SuzukiTrotter
from classiq.builtin_functions.suzuki_trotter import SuzukiParameters
from classiq import Model, synthesize, show

model = Model()
x = sympy.Symbol("x")
y = sympy.Symbol("y")
trotter_params = SuzukiTrotter(
    pauli_operator=PauliOperator(pauli_list=[("XXZ", 1), ("YXZ", 0.5)]),
    evolution_coefficient=(x + y) / 2,
    suzuki_parameters=SuzukiParameters(order=1, repetitions=1),
)
model.SuzukiTrotter(trotter_params)
qprog = synthesize(model.get_model())
show(qprog)

Opening: https://platform.classiq.io/circuit/28e3a4bd-7b8e-48a1-a4b3-f4b787c7889e?version=0.36.1


In [101]:
from sympy import factorial
from classiq import RX, QBit, QFunc, allocate, invert, control, power, show, Output, QNum, within_apply, allocate_num, hadamard_transform, inplace_xor

@QFunc
def symbol_mul(b:QParam[float], x:QNum, output:Output[QNum]) -> None:
    output|= b*x


@QFunc
def inplace_linear_attempt(b: QParam[int], x:QNum, y: QNum):
    tmp = QNum('tmp')
    symbol_mul(factorial((2*b)),x,tmp)
    #symbol_mul(b,x,tmp)
    inplace_xor(tmp,y)

@QFunc
def main(x: Output[QNum],y: Output[QNum]):

    b = 2

    allocate_num(4,False,0,y)
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    inplace_linear_attempt(b,x,y)
qmod = create_model(main)

qprog= synthesize(qmod)
show(qprog)

/home/cuquantum/conda/envs/cuquantum-23.10/lib/python3.10/site-packages/sympy/core/function.py:297: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'2 * (b)'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  args = list(map(sympify, args))


Opening: https://platform.classiq.io/circuit/ce1dc0a2-445a-42d0-aa42-75bf23e7fd0e?version=0.36.1


In [70]:
import sympy
from sympy.core.sympify import kernS
b = sympy.Symbol("b")
exp = b*sympy.factorial(2*b)
sympy.N(exp, 5)

b*factorial(2*b)

In [80]:
import sympy
from classiq import RX, QBit, QFunc, allocate, invert, control, power, show

@QFunc
def foo(a:int, qb: QBit) -> None:
    exp = a*sympy.factorial(2*a)
    RX(exp, qb)


@QFunc
def main() -> None:
    qb1 = QBit("qb1")
    qb2 = QBit("qb2")
    a = 2
    allocate(1, qb1)
    allocate(1, qb2)
    invert(lambda: foo(a, qb1))
    control(lambda: power(2, lambda: foo(a, qb2)), qb1)

qmod = create_model(main)

qprog= synthesize(qmod)
show(qprog)

IllegalParamsError: Unsupported type hint '<class 'int'>' for argument 'a'.
Note - QMOD functions can declare classical parameters using the type hint 'QParam'.
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack
If the error persists feel free to open a ticket at: https://short.classiq.io/support